# 笔记

## 概括(AI 生成+自己完善)

该教程通过一个具体的**图像分类任务**——对 **FashionMNIST 数据集**中的衣物图片进行分类——来引导初学者入门 PyTorch。其核心目标是展示如何利用 PyTorch 完成一个典型的深度学习项目流程：从数据准备、模型构建，到模型训练、评估，以及最终的模型保存与加载。

教程主要介绍了以下几个方面的 PyTorch 核心概念和操作：

- **数据处理与加载**：如何使用 `Dataset` 和 `DataLoader` 来高效地准备和供给数据。
- **模型构建**：如何通过继承 `nn.Module` 来定义神经网络结构，并使用预定义的层 (如线性层、激活函数)。
- **模型训练**：如何定义损失函数、选择优化器，并实现标准的训练循环 (前向传播、计算损失、反向传播、更新参数)。
- **模型评估**：如何在测试集上评估模型性能。
- **模型保存与加载**：如何保存训练好的模型参数，并在需要时重新加载使用。

## 关键函数、语法(自己总结+AI 优化)

- `torch.utils.data.Dataset`: 数据集的抽象基类。自定义数据集时，需继承并实现 `__len__` (返回数据集大小) 和 `__getitem__` (按索引获取数据样本) 两个核心方法。
- `torch.utils.data.DataLoader`: 数据加载器。封装 `Dataset` 以提供数据的迭代访问，支持批处理 (batching)、打乱 (shuffling) 及多进程加载等功能。详细的可以参考官方文档：[https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader](https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)
- `torchvision.datasets`: `torchvision` 库的一部分，提供常用的计算机视觉数据集，如本教程中使用的 `FashionMNIST`。可通过参数指定训练/测试集、下载及预处理转换。
- `torchvision.transforms.ToTensor`: 一种数据预处理转换。将 PIL.Image 或 `numpy.ndarray` (H x W x C) 格式的图像转换为 `torch.Tensor` (C x H x W)，并将像素值从 `[0, 255]` 归一化到 `[0.0, 1.0]`。
- `torch.cuda.is_available()`: 返回布尔值的函数，检测当前系统是否安装并配置了 NVIDIA CUDA 支持，用于判断是否可以将计算迁移到 GPU。
- `torch.nn.Module`: 所有神经网络层的基类。自定义网络模型必须继承此类。核心功能包括参数注册与管理、子模块嵌套、设备迁移 (`.to(device)`) 及状态保存/加载 (`state_dict`, `load_state_dict`)。详细的可以参考官方文档：[https://docs.pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module](https://docs.pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module)
- `__init__(self)` (在 `nn.Module` 子类中): 构造方法。在此处实例化并注册网络层 (如 `nn.Linear`, `nn.Conv2d`) 及其他子模块作为类的属性。掌握了面向对象的相关基础知识就不难理解。
- `forward(self, input_data)` (在 `nn.Module` 子类中): 定义前向传播的计算流程。输入数据 `input_data` 通过在 `__init__` 中定义的网络层和操作，最终返回模型的输出。
- `torch.nn.Flatten(start_dim=1, end_dim=-1)`: 将输入张量从 `start_dim` 到 `end_dim` 的维度展平。常用于将卷积层的多维输出转换为适合全连接层的二维张量 `(batch_size, features)`。此处 `end_dim=-1` 表示展平操作将进行到张量的最后一个维度（包含该维度）。
- `torch.nn.Sequential(*modules)`: 一个有序的模块容器。数据会按照模块在构造函数中传入的顺序依次通过所有模块。是快速构建简单线性堆叠模型的便捷方式。
- `torch.nn.Linear(in_features, out_features)`: 全连接层或线性层。对输入应用线性变换 $y = xW^T + b$。`in_features` 指输入特征维度，`out_features` 指输出特征维度。

![线性层示意图](attachments\Python_Pytorch_nn_Linear_i3_o2_01.png)

- `torch.nn.ReLU()`: 修正线性单元 (Rectified Linear Unit) 激活函数。按元素应用 $f(x) = \max(0, x)$，为模型引入非线性。除此以外，还有多种非线性激活函数：

![non-linear](attachments\non-linear.jfif)

- `module_instance.to(device)`: `nn.Module` 或 `torch.Tensor` 的方法。将模块的所有参数和缓冲区，或张量本身，迁移到指定的计算设备 `device` (如 `'cpu'` 或 `'cuda'`)。对 `nn.Module` 通常是原地操作，即直接对模块本身进行修改。
- `torch.nn.CrossEntropyLoss()`: 交叉熵损失函数。常用于多分类问题。内部集成了 `LogSoftmax` 和 `NLLLoss`，因此输入应为原始 logits (未经 softmax 的分数)，而不是经 softmax 的输出，**易错点**！
- `torch.optim.SGD(model.parameters(), lr=learning_rate)`: 随机梯度下降 (Stochastic Gradient Descent) 优化器。`model.parameters()` 提供模型中所有可训练的参数，`lr` 为学习率。
- `model.parameters()`: `nn.Module` 的方法，返回一个包含模型所有可学习参数 (通常是权重和偏置，即 `requires_grad=True` 的张量) 的迭代器。
- `model.train()`: 将模型设置为训练模式。此模式会启用诸如 `Dropout` 和 `BatchNorm` 等特定层的训练期行为。
- `tensor_instance.to(device)`: `torch.Tensor` 的方法，将张量的数据复制到指定的 `device`。如果张量已在目标设备，则返回自身。
- `optimizer.zero_grad()`: 优化器的方法。在计算每个批次的梯度之前，清除先前累积的梯度。防止梯度在多次反向传播中叠加。
- `loss_tensor.backward()`: 在标量损失张量上调用此方法，以启动反向传播。PyTorch 的自动微分引擎 (Autograd) 会计算损失相对于所有 `requires_grad=True` 的参数的梯度，并将其累积存储在这些参数的 `.grad` 属性中。
- `optimizer.step()`: 优化器的方法。在梯度计算完成 (`loss.backward()`) 后调用，根据选定的优化算法 (如 SGD) 和参数的 `.grad` 中存储的梯度来更新模型参数。
- `scalar_tensor.item()`: 对于仅包含单个元素的张量 (如平均损失值)，返回其对应的 Python 原生数值 (如 `float` 或 `int`)。
- `model.eval()`: 将模型设置为评估（或推理）模式。此模式会禁用诸如 `Dropout` 层的随机失活行为，并使 `BatchNorm` 层使用其在训练期间学习到的固定统计量。
- `with torch.no_grad():`: 上下文管理器。在其作用域内执行的 PyTorch 运算不会构建计算图，也不会计算梯度。用于模型评估或推理阶段，以减少内存消耗并加速计算。
- `output_tensor.argmax(dim)`: `torch.Tensor` 的方法。返回在指定维度 `dim` 上最大元素的索引。常用于从模型的 logits 输出中获取预测类别。
- `torch.save(object_to_save, 'filepath.pth')`: 将 Python 对象 (如模型的 `state_dict` 或整个模型) 序列化并保存到指定路径的文件中。通常使用 `.pth` 或 `.pt` 作为扩展名。
- `model.state_dict()`: `nn.Module` 的方法。返回一个 Python 字典，其中键是参数名，值是参数张量。它包含了模型所有可学习的参数以及持久化的缓冲区 (如 BatchNorm 的运行均值和方差)。
- `torch.load('filepath.pth', weights_only=True)`: 从文件中加载之前由 `torch.save()` 保存的对象。`weights_only=True` (PyTorch 1.13+) 是一个安全选项，一般在仅加载模型权重（即 `state_dict`）时使用，以防止加载恶意代码。
- `model.load_state_dict(loaded_state_dict)`: `nn.Module` 的方法。将从文件加载并可能经过处理的 `state_dict` 中的参数和缓冲区加载到当前模型实例中。默认情况下，`state_dict` 的键必须与模型自身的键完全匹配 (`strict=True`)。

## 提问与解答(自己提问+AI 回答)

- **问题1 (对于 "Working with data" 当中 `DataLoader` 的使用):**
  该教程展示了 `DataLoader` 的基本用法，并设置了 `batch_size`。除了 `batch_size`，`DataLoader` 初始化时还有哪些其他的参数？它们各自的作用是什么，我们一般在哪些情况下需要调整它们？

  - **解答:**
    `DataLoader` 除了 `dataset` 和 `batch_size` 外，还有几个常用且重要的参数：
    1.  `shuffle` (布尔类型, 默认为 `False`): 设置为 `True` 时，会在每个 epoch 开始时打乱数据集的顺序。这有助于模型学习到更鲁棒的特征，防止模型因数据顺序而产生偏见，尤其是在训练集中样本类别分布不均或存在某种序列模式时非常有用。在训练阶段通常建议设置为 `True`，而在验证和测试阶段则应为 `False` 以确保评估的一致性。
    2.  `num_workers` (整型, 默认为 `0`): 指定用于数据加载的子进程数量。当 `num_workers > 0` 时，数据加载将在后台由多个子进程并行处理，这可以显著加速数据准备过程，尤其是在数据预处理比较复杂或磁盘 I/O 成为瓶颈时。CPU 核心数是一个常见的参考值，但最佳值需要根据具体硬件和任务进行调整。如果设置为 `0`，则数据将在主进程中加载。
    3.  `pin_memory` (布尔类型, 默认为 `False`): 如果设置为 `True` 并且正在使用 GPU (CUDA)，`DataLoader` 会将加载的张量数据复制到 CUDA 的固定内存 (pinned memory) 中。这可以加速数据从 CPU 到 GPU 的传输。当 GPU 成为训练瓶颈时，开启此选项可能会有性能提升。
    4.  `drop_last` (布尔类型, 默认为 `False`): 如果数据集大小不能被 `batch_size` 整除，最后一个批次的数据量会小于 `batch_size`。如果设置为 `True`，则这个不完整的批次将被丢弃。在一些需要固定批次大小的场景（例如某些特定的网络结构或分布式训练设置）下，这个参数可能需要设置为 `True`。

    参考资料：
    - PyTorch 官方文档 `torch.utils.data.DataLoader`: [https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

- **问题2 (对于 "Creating Models" 部分 `NeuralNetwork` 定义中 `nn.ReLU` 的使用):**
  该教程的 `NeuralNetwork` 类使用了 `nn.ReLU` 作为激活函数。除了 ReLU，深度学习中还有哪些常见的激活函数？它们各自有什么主要的优点和缺点？为什么我们通常使用 ReLU 而不是其他激活函数？

  - **解答:**
    除了 ReLU (Rectified Linear Unit)，常见的激活函数还包括：
    1.  **Sigmoid (Logistic):**
        -   公式: $ \sigma(x) = \frac{1}{1 + e^{-x}} $
        -   输出范围: (0, 1)
        -   优点: 输出在 (0,1) 之间，可以用作概率表示；平滑，处处可导。
        -   缺点:
            -   **梯度消失**: 当输入值过大或过小时，梯度趋近于0，导致深层网络训练困难。
            -   输出不是零中心的 (zero-centered)，可能影响后续层参数更新的效率。
            -   计算复杂度相对较高 (涉及指数运算)。
        -   常用于二分类问题的输出层或需要输出概率的场景。

    2.  **Tanh (Hyperbolic Tangent):**
        -   公式: $ \tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}} = 2 \cdot \text{sigmoid}(2x) - 1 $
        -   输出范围: (-1, 1)
        -   优点: 输出是零中心的，通常比 Sigmoid 收敛更快。
        -   缺点: 仍然存在梯度消失问题 (虽然比 Sigmoid 略好，因为梯度范围更大)。计算复杂度相对较高。

    3.  **LeakyReLU (Leaky Rectified Linear Unit):**
        -   公式: $ f(x) = \begin{cases} x & \text{if } x > 0 \\ \alpha x & \text{if } x \le 0 \end{cases} $，其中 $ \alpha $ 是一个小的正常数 (如 0.01)。
        -   输出范围: $ (-\infty, \infty) $
        -   优点: 解决了 ReLU 在负输入区间的神经元“死亡”问题 (dead neuron problem)，允许负值区域有小的非零梯度。
        -   缺点: $ \alpha $ 的选择可能需要调整；性能提升不总能保证。

    4.  **ELU (Exponential Linear Unit):**
        -   公式: $ f(x) = \begin{cases} x & \text{if } x > 0 \\ \alpha (e^x - 1) & \text{if } x \le 0 \end{cases} $，其中 $ \alpha > 0 $。
        -   优点: 结合了 ReLU 和 LeakyReLU 的优点，输出均值接近零，具有一定的抗噪声能力。
        -   缺点: 计算复杂度高于 ReLU (涉及指数运算)。

    **为什么 ReLU 是一个良好且常用的默认选择？**
    -   **计算高效**: ReLU 的计算非常简单 (一个 `max(0, x)` 操作)，比 Sigmoid 和 Tanh 的指数运算快得多。
    -   **缓解梯度消失**: 在正数区间，ReLU 的梯度恒为1，这有助于梯度在网络中传播，从而在一定程度上缓解了梯度消失问题，使得训练更深的网络成为可能。
    -   **稀疏性**: ReLU 会将负输入置为0，这使得网络中的一些神经元输出为0，从而引入了稀疏性。这种稀疏性可以使网络更高效，并且可能具有更好的泛化能力（类似于正则化的效果）。

    尽管 ReLU 有“死亡神经元” (当输入恒为负时，神经元不再激活且梯度为0) 的问题，但在实践中，通过合理的初始化和学习率设置，ReLU 通常表现良好，并且是许多现代神经网络架构的默认激活函数。

    参考资料：
    - "Deep Learning" book by Goodfellow, Bengio, and Courville (Chapter 6 on Deep Feedforward Networks, specifically activation functions).
    - Stanford CS231n Course Notes on Activation Functions: [https://cs231n.github.io/neural-networks-1/#actfun](https://cs231n.github.io/neural-networks-1/#actfun)
    - PyTorch documentation on non-linear activations: [https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)

- **问题3 (关于 "Optimizing the Model Parameters" 部分优化器的选择和学习率的设置):**
  该教程使用了 `torch.optim.SGD` 优化器，并设置了固定的学习率 `lr=1e-3`。对此，我有以下几个问题：
    1. 如果学习率设置得过大或过小，分别可能会导致什么问题？
    2. 除了 SGD，还有哪些在实践中常用的优化器？它们相对于 SGD 有哪些改进？
    3. 教程中使用了固定的学习率，在更复杂的任务中，我们可能会使用学习率调度器 (Learning Rate Scheduler)。它的作用是什么？你能否举例说明一种常见的学习率调整策略？

  - **解答:**
    1. **学习率的影响:**
        -   **学习率过大**: 可能导致优化算法在损失函数的最小值附近“震荡”甚至“发散”，无法收敛到最优解。想象一下你下山，步子迈得太大，可能会直接跨过山谷的最低点，甚至跑到对面的山坡上去了。
        -   **学习率过小**: 可能导致收敛速度非常缓慢，需要大量的训练迭代才能达到较好的性能。同时，也更容易陷入局部最小值 (local minima) 或鞍点 (saddle points)。想象一下你下山，每一步都迈得非常小，虽然最终能到山脚，但会花费非常长的时间。

    2. **其他常用优化器及其改进:**
        除了 SGD (随机梯度下降)，还有许多更先进的优化器，它们通常通过自适应地调整每个参数的学习率或引入动量 (momentum) 来加速收敛并改善 SGD 的一些缺点。
        1.  **SGD with Momentum (带动量的SGD):**
            -   改进: 在 SGD 的基础上引入了动量项，该动量项累积了历史梯度的方向。这有助于加速在梯度方向一致的维度上的学习，并抑制震荡，使得优化路径更平滑，更容易跳出局部最优。
            -   PyTorch 实现: `torch.optim.SGD(params, lr, momentum=0.9)` (momentum 通常设为0.9左右)。
        2.  **Adagrad (Adaptive Gradient Algorithm):**
            -   改进: 对每个参数使用不同的学习率。对于不经常更新的参数，它会使用较大的学习率；对于经常更新的参数，则使用较小的学习率。适合处理稀疏数据。
            -   缺点: 学习率会单调递减，可能导致学习率过早变得太小，从而提前停止学习。
            -   PyTorch 实现: `torch.optim.Adagrad(params, lr)`
        3.  **RMSprop (Root Mean Square Propagation):**
            -   改进: Adagrad 的一个改进版本，通过引入一个衰减系数来解决 Adagrad 学习率急剧下降的问题。它不累积所有历史平方梯度，而是关注最近一段时间的梯度大小。
            -   PyTorch 实现: `torch.optim.RMSprop(params, lr)`
        4.  **Adam (Adaptive Moment Estimation):**
            -   改进: 结合了 Momentum 和 RMSprop 的思想。它既计算了梯度的一阶矩估计 (均值，即动量项)，也计算了梯度的二阶矩估计 (方差的无偏估计，用于自适应调整学习率)。Adam 通常被认为在各种任务和网络结构上都表现良好，并且对初始学习率的选择不如 SGD 敏感，是目前非常流行和常用的优化器之一。
            -   PyTorch 实现: `torch.optim.Adam(params, lr)`

    3. **学习率调度器 (Learning Rate Scheduler):**
        - 学习率调度器的作用是在训练过程中动态地调整学习率。通常在训练开始时使用较大的学习率以快速接近最优解，然后在训练后期逐渐减小学习率，以便更精细地调整参数，帮助模型收敛到更好的局部最优值并避免震荡。
        - 一种常见的学习率调整策略是 **StepLR (阶梯式衰减)**：
            -   工作原理: 在预设的训练轮数 (epoch) 间隔 (`step_size`) 后，将当前学习率乘以一个衰减因子 (`gamma`)。
            -   例如，初始学习率为 0.1，`step_size=30`，`gamma=0.1`。那么在第30个 epoch 结束后，学习率会变成 0.01；在第60个 epoch 结束后，学习率会变成 0.001，以此类推。
            -   PyTorch 实现: `torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)`
        - 其他常见的策略还包括 `MultiStepLR` (在指定的多个 epoch 处衰减学习率)，`ExponentialLR` (指数衰减)，以及 `CosineAnnealingLR` (余弦退火，学习率按余弦函数周期性变化) 等。

    参考资料：
    - "An overview of gradient descent optimization algorithms" by Sebastian Ruder: [https://ruder.io/optimizing-gradient-descent/](https://ruder.io/optimizing-gradient-descent/) (这篇博客文章对各种优化器有非常好的总结)
    - PyTorch 官方文档 `torch.optim`: [https://pytorch.org/docs/stable/optim.html](https://pytorch.org/docs/stable/optim.html) (包含了所有优化器和学习率调度器的详细信息)


[Learn the Basics](intro.html) \|\| **Quickstart** \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Quickstart
==========

This section runs through the API for common tasks in machine learning.
Refer to the links in each section to dive deeper.

Working with data
-----------------

PyTorch has two [primitives to work with
data](https://pytorch.org/docs/stable/data.html):
`torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. `Dataset`
stores the samples and their corresponding labels, and `DataLoader`
wraps an iterable around the `Dataset`.


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as
[TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and
[TorchAudio](https://pytorch.org/audio/stable/index.html), all of which
include datasets. For this tutorial, we will be using a TorchVision
dataset.

The `torchvision.datasets` module contains `Dataset` objects for many
real-world vision data like CIFAR, COCO ([full list
here](https://pytorch.org/vision/stable/datasets.html)). In this
tutorial, we use the FashionMNIST dataset. Every TorchVision `Dataset`
includes two arguments: `transform` and `target_transform` to modify the
samples and labels respectively.


In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

We pass the `Dataset` as an argument to `DataLoader`. This wraps an
iterable over our dataset, and supports automatic batching, sampling,
shuffling and multiprocess data loading. Here we define a batch size of
64, i.e. each element in the dataloader iterable will return a batch of
64 features and labels.


In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Read more about [loading data in PyTorch](data_tutorial.html).


------------------------------------------------------------------------


Creating Models
===============

To define a neural network in PyTorch, we create a class that inherits
from
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
We define the layers of the network in the `__init__` function and
specify how data will pass through the network in the `forward`
function. To accelerate operations in the neural network, we move it to
the
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [4]:
# device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

# 检查是否有可用的 CUDA (NVIDIA GPU)
if torch.cuda.is_available():
    device = "cuda"
# 如果没有，则使用 CPU
else:
    device = "cpu"

print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Read more about [building neural networks in
PyTorch](buildmodel_tutorial.html).


------------------------------------------------------------------------


Optimizing the Model Parameters
===============================

To train a model, we need a [loss
function](https://pytorch.org/docs/stable/nn.html#loss-functions) and an
[optimizer](https://pytorch.org/docs/stable/optim.html).


In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training
dataset (fed to it in batches), and backpropagates the prediction error
to adjust the model\'s parameters.


In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model\'s performance against the test dataset to
ensure it is learning.


In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*).
During each epoch, the model learns parameters to make better
predictions. We print the model\'s accuracy and loss at each epoch;
we\'d like to see the accuracy increase and the loss decrease with every
epoch.


In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


loss: 2.305108  [   64/60000]
loss: 2.298959  [ 6464/60000]
loss: 2.283111  [12864/60000]
loss: 2.279013  [19264/60000]
loss: 2.248929  [25664/60000]
loss: 2.232475  [32064/60000]
loss: 2.238815  [38464/60000]
loss: 2.204337  [44864/60000]
loss: 2.202091  [51264/60000]
loss: 2.178800  [57664/60000]
Test Error: 
 Accuracy: 44.1%, Avg loss: 2.171968 

Epoch 2
-------------------------------
loss: 2.178049  [   64/60000]
loss: 2.175273  [ 6464/60000]
loss: 2.122789  [12864/60000]
loss: 2.136841  [19264/60000]
loss: 2.080745  [25664/60000]
loss: 2.036564  [32064/60000]
loss: 2.058276  [38464/60000]
loss: 1.980273  [44864/60000]
loss: 1.982165  [51264/60000]
loss: 1.926575  [57664/60000]
Test Error: 
 Accuracy: 51.2%, Avg loss: 1.917008 

Epoch 3
-------------------------------
loss: 1.944713  [   64/60000]
loss: 1.924238  [ 6464/60000]
loss: 1.804566  [12864/60000]
loss: 1.845308  [19264/60000]
loss: 1.741824  [25664/60000]
loss: 1.695971  [32064/60000]
loss: 1.721411  [38464/60000]
loss: 

Read more about [Training your model](optimization_tutorial.html).


------------------------------------------------------------------------


Saving Models
=============

A common way to save a model is to serialize the internal state
dictionary (containing the model parameters).


In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Loading Models
==============

The process for loading a model includes re-creating the model structure
and loading the state dictionary into it.


In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

This model can now be used to make predictions.


In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


Read more about [Saving & Loading your
model](saveloadrun_tutorial.html).
